In [38]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import re

In [39]:
data = pd.read_csv("D:python/tahrirchi/tahrirchi/data.csv")

In [56]:
data.head(5)

,source_url,access_datetime,content
0,https://kun.uz/uz/news/2022/12/01/oz-joniga-qa...,2022-12-01 20:13:46.283372,O‘zbekiston musulmonlari idorasi raisining bir...
1,https://kun.uz/uz/news/2022/12/01/kocha-qonunl...,2022-12-01 20:13:47.683332,Yaqinda Namangan shahridagi “Mulla qirg‘iz” ma...
2,https://kun.uz/uz/news/2022/12/01/sudlanayotga...,2022-12-01 20:13:49.090811,«Huquqiy tayanch» birlashmasi rahbari A’zam Fa...
3,https://kun.uz/uz/news/2022/11/30/ozekspomarka...,2022-12-01 20:13:50.487216,Bugun O‘zekspomarkazda O‘zbekiston Respublikas...
4,https://kun.uz/uz/news/2022/11/30/turonbank-ha...,2022-12-01 20:13:51.970941,"O‘lkamizda iqlim sharoitlari o‘zgarib, sovuq k..."


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   source_url       990 non-null    object
 1   access_datetime  990 non-null    object
 2   content          989 non-null    object
dtypes: object(3)
memory usage: 23.3+ KB


In [58]:
data = data.dropna()

In [45]:
data.rename(columns={'access_datetime ': 'access_datetime', 'content ': 'content'}, inplace=True)

In [136]:
# lowercase_ function makes all words in lowercase
def lowercase_(df):
    df = df.apply(lambda x: x.astype(str).str.lower())
    return df

lowercase = FunctionTransformer(lowercase_)

In [137]:
# remove_chars_ function removes all unnecessary chars
def remove_chars_(df):
    df['content'] = df['content'].map(lambda x: re.sub(r'([0-9]+)', r' \1 ', x))
    df['content'] = df['content'].map(lambda x: re.sub(r"[^0-9a-z‘’]+",r" ", x))
    return df

remove_chars = FunctionTransformer(remove_chars_)

In [138]:
def add_word_column_(df):
    values = []
    for index in range(len(df)):
        values.append(' '.join(df.iloc[index]['content'].split()))
    df['word'] = values
    return df

add_word_column = FunctionTransformer(add_word_column_)

In [145]:
def count_word_(df):
    values = []
    for index in range(len(df)):
        values.append(len(df.iloc[index]['word']))
    df['word_count'] = values
    return df
    
count_word = FunctionTransformer(count_word_)

In [146]:
pipeline = Pipeline(
    [
        ('lowercase', lowercase),
        ('remove_chars', remove_chars),
        ('add_word_column', add_word_column),
        ('count_word', count_word)
    ]
)

In [150]:
modified = pipeline.transform(data)

In [152]:
modified.head(5)

,source_url,access_datetime,content,word,word_count
0,https://kun.uz/uz/news/2022/12/01/oz-joniga-qa...,2022-12-01 20:13:46.283372,o‘zbekiston musulmonlari idorasi raisining bir...,o‘zbekiston musulmonlari idorasi raisining bir...,2321
1,https://kun.uz/uz/news/2022/12/01/kocha-qonunl...,2022-12-01 20:13:47.683332,yaqinda namangan shahridagi mulla qirg‘iz madr...,yaqinda namangan shahridagi mulla qirg‘iz madr...,7813
2,https://kun.uz/uz/news/2022/12/01/sudlanayotga...,2022-12-01 20:13:49.090811,huquqiy tayanch birlashmasi rahbari a’zam far...,huquqiy tayanch birlashmasi rahbari a’zam farm...,2424
3,https://kun.uz/uz/news/2022/11/30/ozekspomarka...,2022-12-01 20:13:50.487216,bugun o‘zekspomarkazda o‘zbekiston respublikas...,bugun o‘zekspomarkazda o‘zbekiston respublikas...,1767
4,https://kun.uz/uz/news/2022/11/30/turonbank-ha...,2022-12-01 20:13:51.970941,o‘lkamizda iqlim sharoitlari o‘zgarib sovuq ku...,o‘lkamizda iqlim sharoitlari o‘zgarib sovuq ku...,3989


In [153]:
modified.to_csv('modified.csv')